In [1]:
import pandas as pd
import os

from formatacao_sumulas import FootballDataProcessor
from algorithm import FootballMatchRating
import numpy as np
import warnings
warnings.filterwarnings("ignore")


id_players = pd.read_excel('id_players.xlsx', index_col=0)


data_folder = '../Data/'
file_names = ['Serie_A']


In [2]:
years = range(2013, 2024)


for year in years:
    if year == 2016:
        pass
    else:
        print(year)
        df_jogos_2014 = pd.read_json(f'../Data/Serie_A_{year}_games.json')
        df_jogos_2014_ = df_jogos_2014.T


        for index, row in df_jogos_2014_.iterrows():
            processor = FootballDataProcessor(df_jogos_2014_, index-1)
            final_df = processor.process(data_folder, file_names, year)
            final_df.rename(columns={'player_id': 'player'}, inplace=True)

            final_df['player'] = final_df['player'].astype('string')
            id_players['player'] = id_players['player'].astype('string')

            final_df_ = pd.merge(final_df, id_players, on='player', how='left')

            team_1_name = df_jogos_2014_.iloc[index-1,0]
            team_2_name = df_jogos_2014_.iloc[index-1,1]


            # Criar uma instância de FootballMatchRating
            rating_calculator = FootballMatchRating(final_df_, team_1_name, team_2_name)

            # Atualizar as ratings
            updated_match_data = rating_calculator.update_ratings()
            # Garantir que 'player' em updated_match_data está como string
            updated_match_data['player'] = updated_match_data['player'].astype('string')
            # Merge para obter apenas as colunas 'rating' e 'games_played' que precisam ser atualizadas
            updated_match_data.rename(columns={'player_name_x': 'player_name'}, inplace=True)
            updated_match_data.rename(columns={'time_jogador_x': 'time_jogador'}, inplace=True)

            updated_values = id_players[['player']].merge(
                updated_match_data[['player', 'rating', 'games_played', 'player_name', 'time_jogador']],
                on='player',
                how='left'
            )

            # Atualizar apenas as linhas corespondentes em id_players
            id_players.update(updated_values)

id_players


2013
[(47, 'Home')]
[(27, 'Away'), (53, 'Home'), (15, 'Home')]
[(25, 'Away'), (73, 'Home')]
[(2, 'Home'), (11, 'Home'), (30, 'Away'), (63, 'Away')]
[(15, 'Home'), (70, 'Home')]
[(8, 'Away'), (44, 'Away')]
[(45, 'Home'), (46, 'Home'), (72, 'Away'), (80, 'Home')]
[(6, 'Home'), (31, 'Home'), (40, 'Home'), (42, 'Home'), (80, 'Home')]
[(49, 'Away'), (53, 'Home'), (45, 'Home')]
2
[]
[(14, 'Home'), (21, 'Home'), (71, 'Away')]
2
[(26, 'Away'), (68, 'Away')]
[(60, 'Home'), (64, 'Home'), (72, 'Home'), (74, 'Home'), (82, 'Away'), (84, 'Home')]
[]
[(13, 'Home'), (16, 'Home')]
[(36, 'Home'), (86, 'Away')]
[(13, 'Away'), (45, 'Away'), (49, 'Away')]
[(45, 'Away'), (59, 'Away')]
[(6, 'Away'), (28, 'Away'), (43, 'Away'), (46, 'Away')]
[(22, 'Home'), (83, 'Home'), (36, 'Away')]
[(6, 'Home'), (25, 'Away'), (56, 'Home')]
[(25, 'Home'), (33, 'Home'), (80, 'Home')]
[(73, 'Home')]
[(31, 'Home'), (59, 'Home')]
[(10, 'Away'), (61, 'Away'), (64, 'Home')]
[(33, 'Away'), (75, 'Home')]
[(15, 'Home'), (31, 'Away'),

,player,rating,age,games_played,player_name,time_jogador
0,293092,1500.0,25,50,NaN,NaN
1,554797,1500.0,25,50,NaN,NaN
2,413816,1500.0,25,50,NaN,NaN
3,508147,1500.0,25,50,NaN,NaN
4,584599,1500.0,25,50,NaN,NaN
...,...,...,...,...,...,...
3082,346555,1500.0,25,50,NaN,NaN
3083,299156,1500.0,25,50,NaN,NaN
3084,319981,1500.0,25,50,NaN,NaN
3085,693536,1500.0,25,50,NaN,NaN


In [3]:
x = id_players.sort_values(by='rating', ascending=False)
x.to_excel('teste.xlsx')
x

,player,rating,age,games_played,player_name,time_jogador
1539,177749,1645.376721,25,81,26Dede Anderson Vital da Silva,Cruzeiro / MG
1816,129292,1624.388203,25,86,1Fabio Fabio Deivson Lopes ...,Cruzeiro / MG
2729,163518,1609.715555,25,80,19Nilton Nilton Ferreira Junior,Cruzeiro / MG
127,157280,1602.560195,25,85,6Egidio Egidio de Araujo Per ...,Cruzeiro / MG
577,320111,1601.710407,25,73,16Lucas Lucas Silva Borges,Cruzeiro / MG
...,...,...,...,...,...,...
1744,173742,1364.813577,25,73,6Willian William Augusto Alve...,Náutico / PE
2635,166491,1360.852959,25,79,8Derley Wanderley de Jesus Sousa,Náutico / PE
1492,135462,1341.543263,25,80,1Ricardo Ricardo Ferreira Berna,Náutico / PE
2839,177998,1339.559702,25,74,6Bruno Bruno Bairros Collaco,Náutico / PE
